In [3]:
import os
import pickle
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,classification_report

# Modélisation

In [4]:
print(os.path.abspath(''))
file_path = input('Chemin absolu du fichier à charger : \n')
# data_avec_etiquettes.txt

C:\Users\trian\Desktop\1.0___Data_mining\Projet


Chemin absolu du fichier à charger : 
 data_avec_etiquettes.txt


In [5]:
# Variables pertinentes
cols = ['V14', 'V84', 'V96', 'V159', 'V160', 'V161','V162','V163', 
        'V164', 'V165', 'V166', 'V167', 'V168', 'V169', 'V170', 
        'V171', 'V172', 'V173', 'V174', 'V175', 'V176', 'V177', 
        'V180', 'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 
        'V187', 'V188', 'V189', 'V190', 'V191', 'V192', 'V193', 
        'V194', 'V195', 'V197', 'V198', 'V199','V200']

# variables explicatives
X = pd.read_csv(file_path,delimiter='\t',usecols=cols
[:-1])

# variable cible  
y = pd.read_csv(file_path,delimiter='\t',usecols=[cols
[-1]])   

y=y.to_numpy().reshape(-1)

# recodage des variables qualitatives
X=pd.get_dummies(X, columns=['V160','V162'],drop_first=True)
X[["V161"]]=X[["V161"]].apply(LabelEncoder().fit_transform) 

# Division apprentissage/test
X_train,X_test,y_train,y_test=model_selection.train_test_split(X,y,
test_size=0.3,stratify=y,random_state=42)

# Apprentissage sur le train set
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train) # met un peu de temps

DecisionTreeClassifier()

Génération du modèle

In [ ]:
#f=open("dtree.sav","wb")
#pickle.dump(dtree,f)
#f.close()

In [6]:
f=open("dtree.sav","rb")
dtree=pickle.load(f)
f.close()

In [7]:
pred=dtree.predict(X_test)

# Evaluation

In [8]:
df=pd.DataFrame({'y_test':pd.Series(y_test).value_counts(),'pred':pd.Series(pred).value_counts()})
df=df.fillna(0)
df=df.astype(int)
df

,y_test,pred
m1,661,661
m10,32160,32160
m11,69,68
m12,29184,29188
m13,1,1
m14,1,1
m15,79,79
m16,312,311
m17,3,5
m18,477,478


In [9]:
print(df[df['y_test']==df['pred']].shape[0])

10


In [10]:
df=pd.DataFrame({'y_test':y_test,'prediction':pred})
df[df['y_test']!=df['prediction']].shape[0]

22

In [11]:
# Taux du succès
print("Taux de succès : ",accuracy_score(y_test,pred))

#Calcul du taux d'erreur
print("Taux d'erreur : "+str(1.0-accuracy_score(y_test,pred)))

Taux de succès :  0.9998515589681999
Taux d'erreur : 0.00014844103180011725


In [12]:
labels=list(set(y_test))
labels.sort()
precision = pd.DataFrame({"precision_score":precision_score(y_test,pred,average=None,labels=labels)},index=labels)
precision

,precision_score
m1,1.000000
m10,0.999969
m11,1.000000
m12,0.999657
m13,1.000000
m14,1.000000
m15,1.000000
m16,0.996785
m17,0.600000
m18,0.995816


In [13]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

          m1       1.00      1.00      1.00       661
         m10       1.00      1.00      1.00     32160
         m11       1.00      0.99      0.99        69
         m12       1.00      1.00      1.00     29184
         m13       1.00      1.00      1.00         1
         m14       1.00      1.00      1.00         1
         m15       1.00      1.00      1.00        79
         m16       1.00      0.99      1.00       312
         m17       0.60      1.00      0.75         3
         m18       1.00      1.00      1.00       477
         m19       1.00      1.00      1.00     84237
          m2       1.00      0.89      0.94         9
         m20       1.00      1.00      1.00         1
         m21       1.00      1.00      1.00       294
         m22       1.00      0.99      1.00       306
         m23       0.83      0.83      0.83         6
          m3       0.33      0.50      0.40         2
          m4       0.89    

In [14]:
#importance des variables
imp = {"VarName":X_train.columns,"Importance":dtree.feature_importances_}
print(pd.DataFrame(imp).sort_values(by="Importance",ascending=False))

     VarName  Importance
22      V182    0.604840
27      V187    0.326148
5       V163    0.022198
33      V193    0.011601
35      V195    0.007291
36      V197    0.006323
8       V166    0.006170
46   V162_m5    0.004644
10      V168    0.001696
4       V161    0.001481
6       V164    0.001348
34      V194    0.001256
31      V191    0.001183
47   V162_m6    0.000660
37      V198    0.000579
21      V181    0.000443
28      V188    0.000354
3       V159    0.000297
11      V169    0.000257
39   V160_m2    0.000255
43   V162_m2    0.000239
32      V192    0.000149
14      V172    0.000067
30      V190    0.000062
41  V162_m10    0.000054
24      V184    0.000052
17      V175    0.000047
13      V171    0.000042
23      V183    0.000039
29      V189    0.000039
12      V170    0.000034
40   V160_m3    0.000030
16      V174    0.000029
18      V176    0.000023
38      V199    0.000015
25      V185    0.000014
1        V84    0.000010
26      V186    0.000010
7       V165    0.000009


In [15]:
from sklearn.tree.export import export_text
r=export_text(dtree,feature_names = list(X_train.columns))
print(r)

|--- V182 <= 321.00
|   |--- V187 <= 0.33
|   |   |--- V193 <= 0.15
|   |   |   |--- V163 <= 0.50
|   |   |   |   |--- V194 <= 0.06
|   |   |   |   |   |--- V188 <= 0.93
|   |   |   |   |   |   |--- V194 <= 0.02
|   |   |   |   |   |   |   |--- class: m10
|   |   |   |   |   |   |--- V194 >  0.02
|   |   |   |   |   |   |   |--- V192 <= 0.06
|   |   |   |   |   |   |   |   |--- class: m16
|   |   |   |   |   |   |   |--- V192 >  0.06
|   |   |   |   |   |   |   |   |--- class: m10
|   |   |   |   |   |--- V188 >  0.93
|   |   |   |   |   |   |--- V184 <= 0.50
|   |   |   |   |   |   |   |--- class: m18
|   |   |   |   |   |   |--- V184 >  0.50
|   |   |   |   |   |   |   |--- class: m10
|   |   |   |   |--- V194 >  0.06
|   |   |   |   |   |--- V193 <= 0.02
|   |   |   |   |   |   |--- class: m10
|   |   |   |   |   |--- V193 >  0.02
|   |   |   |   |   |   |--- class: m16
|   |   |   |--- V163 >  0.50
|   |   |   |   |--- V163 <= 10.00
|   |   |   |   |   |--- V182 <= 20.50
|   |   | 

C:\Users\trian\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.tree.export module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
  warnings.warn(message, FutureWarning)
